In [1]:
#importing the necessary packages
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
#Reading in each csv file for the years of customer data
Customer2014 = pd.read_csv('East Rock_Customer_List_ALL_2014.csv', encoding = 'cp1252')
Customer2015 = pd.read_csv('East Rock_Customer_List_ALL_2015.csv', encoding = 'cp1252')

In [3]:
#Combining the six pandas DataFrames into one using the concatenate function
data = pd.concat([Customer2014, Customer2015], axis=0)

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [4]:
#Spliting the Created column into two rows with the date and time as seperate columns
data['date_created'], data['time_created'] = data['Created'].str.split(' ', 1).str

In [5]:
print(data.head())

   #Prod                   Address                Agent Label +ID  \
0    1.0  10 Marshall ST , Apt 7-E              Hospital (109465)   
1    1.0         115 Van Meter Dr              Newspaper (109455)   
2    1.0           93 South Street   Angies List - Forte (109439)   
3    1.0              19 Myrtle St             Newspaper (109455)   
4    1.0            106 edbert st   Client Referral 2322 (109402)   

      CUSTOM Cancel Reason       City         Created First Name         ID  \
0  020 - Financial Reasons  Irvington   8/12/14 16:52       Iola  657938060   
1  020 - Financial Reasons   Amherst    6/16/14 15:40      Leeta  657847231   
2  020 - Financial Reasons      Barre   7/18/14 13:48    Kathryn  657897142   
3  020 - Financial Reasons  Chicoppee  10/30/14 20:01   Patricia  658086304   
4  020 - Financial Reasons   chicopee   7/22/14 17:08    Paticia  657903565   

    Income Last Name     ...                Phone  Sales Agent  \
0  55490.0    Glover     ...       (973) 375

In [6]:
data['date_created'] = pd.to_datetime(data['date_created'])
data['Last Pay Date'] = pd.to_datetime(data['Last Pay Date'])
data['days'] = (data['Last Pay Date'] - data['date_created']).dt.days
data['months'] = data['days']/30
data.head()

,#Prod,Address,Agent Label +ID,CUSTOM Cancel Reason,City,Created,First Name,ID,Income,Last Name,...,Stage,State,Status,Track Date Cancel Requested,Zip Code,ZipCodeNew,date_created,time_created,days,months
0,1.0,"10 Marshall ST , Apt 7-E",Hospital (109465),020 - Financial Reasons,Irvington,8/12/14 16:52,Iola,657938060,55490.0,Glover,...,Call Center Notified,NJ,Decline Cancel,1/15/15,7111,7111,2014-08-12,16:52,22.0,0.733333
1,1.0,115 Van Meter Dr,Newspaper (109455),020 - Financial Reasons,Amherst,6/16/14 15:40,Leeta,657847231,77857.0,Bailey,...,NaN,MA,Decline Cancel,10/16/14,1002,1002,2014-06-16,15:40,17.0,0.566667
2,1.0,93 South Street,Angies List - Forte (109439),020 - Financial Reasons,Barre,7/18/14 13:48,Kathryn,657897142,95647.0,Hazard,...,NaN,MA,Decline Cancel,1/15/15,1005,1005,2014-07-18,13:48,18.0,0.600000
3,1.0,19 Myrtle St,Newspaper (109455),020 - Financial Reasons,Chicoppee,10/30/14 20:01,Patricia,658086304,35339.0,Stearly,...,Call Center Notified,MA,Decline Cancel,1/15/15,1013,1013,2014-10-30,20:01,2.0,0.066667
4,1.0,106 edbert st,Client Referral 2322 (109402),020 - Financial Reasons,chicopee,7/22/14 17:08,Paticia,657903565,58782.0,jimmo,...,NaN,MA,Decline Cancel,10/16/14,1020,1020,2014-07-22,17:08,43.0,1.433333


In [9]:
# Creating an empty list for column names
names = []

# Creating an empty list for the number of null values in each column
values = []

# Checking for Missing Values
for col in data.columns:
    names.append(col)
    values.append(data[col].isnull().sum())
    print(names[-1],values[-1])

#Prod 29
Address 1
Agent Label +ID 0
CUSTOM Cancel Reason 13048
City 2
Created 0
First Name 0
ID 0
Income 10993
Last Name 1
Last Pay Amount 4248
Last Pay Complete 4248
Last Pay Date 4248
Last Pay Status 4248
Last Pay Type 4442
Pay Method 1750
Phone 36446
Sales Agent 56613
Stage 13891
State 0
Status 0
Track Date Cancel Requested 10554
Zip Code 0
ZipCodeNew 0
date_created 0
time_created 0
days 4248
months 4248


In [10]:
data = data[pd.notnull(data['Last Pay Date'])]

In [11]:
# Creating an empty list for column names
names = []

# Creating an empty list for the number of null values in each column
values = []

# Checking for Missing Values
for col in data.columns:
    names.append(col)
    values.append(data[col].isnull().sum())
    print(names[-1],values[-1])

#Prod 0
Address 1
Agent Label +ID 0
CUSTOM Cancel Reason 11131
City 2
Created 0
First Name 0
ID 0
Income 10385
Last Name 1
Last Pay Amount 0
Last Pay Complete 0
Last Pay Date 0
Last Pay Status 0
Last Pay Type 194
Pay Method 19
Phone 33053
Sales Agent 52799
Stage 11351
State 0
Status 0
Track Date Cancel Requested 10549
Zip Code 0
ZipCodeNew 0
date_created 0
time_created 0
days 0
months 0


In [13]:
data['Status'].unique()

array(['Decline Cancel', 'Returned', 'Duplicate', 'Returned <30 days',
       'Order Canceled', 'Active', 'Decline', 'Returned < 90 days',
       'Restricted', 'Suspend', 'Deactivated', 'Daily Decline',
       'Returned <60 Days', 'Collections', 'Returned to Sender',
       'In Process', 'Chargeback Received'], dtype=object)

In [15]:
data = data.drop(['Address', 'Sales Agent', 'First Name', 'Last Name', 'Phone', 'Zip Code', 'Track Date Cancel Requested'], axis=1)

In [ ]:
data.to_csv("data.csv",",")